In [1]:
# Import Data Preprocessing and Wrangling libraries
import re
from tqdm.notebook import tqdm
import pandas as pd 
import numpy as np
from datetime import datetime

# Import NLP Libraries
import nltk

# Import Visualization Libraries
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import random 
import seaborn as sns

# Downloading periphrals
nltk.download('stopwords')
stop_words = stopwords.words('english')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/xingyuchen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# define function to clean tweets
def clean_tweet(tweet):
    if type(tweet) == np.float:
        return ""
    # lowercase all the letters
    temp = tweet.lower()
    
    temp = re.sub("'", "", temp) # to avoid removing contractions in english
    
    # remove hashtages and mentions
    temp = re.sub("@[A-Za-z0-9_]+","", temp)
    temp = re.sub("#[A-Za-z0-9_]+","", temp)
    
    # remove links
    temp = re.sub(r'http\S+', '', temp)
    temp = re.sub(r"www.\S+", "", temp)
    
    # remove punctuations
    temp = re.sub('[()!?]', ' ', temp)
    temp = re.sub('\[.*?\]',' ', temp)
    
    # flitering non-alphanumeric characters
    temp = re.sub("[^a-z0-9]"," ", temp)
    
    # remove stop word
    temp = temp.split()
    temp = [w for w in temp if not w in stop_words]
    
    # put text into a list 
    temp = " ".join(word for word in temp)
    return temp

In [3]:
# import dataset and check
df = pd.read_csv('./data/prochoice_prolife.csv')
df.head()

,Unnamed: 0,author_id,author_name,author_username,created_at,id,public_metrics,text,retweet_count,like_count,target
0,28588,73506221,Oregon Right to Life,OR_RTL,2022-06-23 00:00:06+00:00,1539760170900799490,"{'retweet_count': 5, 'reply_count': 0, 'like_c...",We know that unsupported pregnancies can gener...,5,13,1
1,28587,96631851,αιяgσ*мαтι¢*σρтιχ,sacraficial,2022-06-23 00:01:14+00:00,1539760456977395712,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",If you are murdered by a George W. Bush weapon...,0,0,1
2,28586,3041829701,skb,skb37027,2022-06-23 00:04:30+00:00,1539761277702213633,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",.@MarshaBlackburn so much for #ProLife. https:...,0,0,1
3,28537,177260708,Right To Life League,Right2LifeLg,2022-06-23 00:09:50+00:00,1539762620055552001,"{'retweet_count': 6, 'reply_count': 3, 'like_c...",Follow our instagram for more posts like this!...,6,19,1
4,27450,1523376591052582912,No Forced Birth,NoForcedBirth,2022-06-23 00:09:57+00:00,1539762647922421763,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",Another classy tweet from the forced birth cro...,0,1,0


In [4]:
# Select useful columns
df = df[['created_at', 'text', 'retweet_count', 'like_count', 'target']]
df.head()

,created_at,text,retweet_count,like_count,target
0,2022-06-23 00:00:06+00:00,We know that unsupported pregnancies can gener...,5,13,1
1,2022-06-23 00:01:14+00:00,If you are murdered by a George W. Bush weapon...,0,0,1
2,2022-06-23 00:04:30+00:00,.@MarshaBlackburn so much for #ProLife. https:...,0,0,1
3,2022-06-23 00:09:50+00:00,Follow our instagram for more posts like this!...,6,19,1
4,2022-06-23 00:09:57+00:00,Another classy tweet from the forced birth cro...,0,1,0


In [ ]:
# Transform 'created_at' column into datetime format
df['original_text'] = df['text']
df['datetime'] = df["created_at"]
df = data.drop('created_at', axis=1)
df['datetime'] = data['datetime'].astype(str).apply(lambda x: x.split('+')[0])
df['datetime'] = data.datetime.apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
df